In [ ]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from matplotlib import pyplot as plt
import seaborn as sns 
%matplotlib inline 
import os
import matplotlib as mpl
from pandas import Series
import re
#Pas de message d'alertes
import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.
df = pd.read_csv("../input/train.csv")  
xtest=pd.read_csv('../input/xtest.csv')


#Lire 5 ligne de manière aléa 
df.sample(5)



#Regardons la taille  de notre dataset
df.dtypes



#convertir le type de author en object 
df['author']=df['author'].astype('object')

df.columns


#Nous allons supprimer la variables entities 
df.drop(columns=['entities', 'Unnamed: 0'], inplace=True)



#Regardons les vleurs manquantes 
df.isnull().sum()



#Statistiques descriptives des variables quantitatives 
df_qual=df.select_dtypes(exclude=['object'])
df_qual.describe().plot(kind = "area",fontsize=22, figsize = (18,8), table = True,colormap="rainbow")
plt.xlabel('',)
plt.ylabel('Value')
plt.title("Statistiques générales des variables ")

df1=df
df1['author']=df1['author'].astype('object')
df1['author'][df1['author'] == '0' ] = 'Trump'
df1['author'][df1['author'] == '1' ] = 'Clinton'
f,ax=plt.subplots(1,2,figsize=(16,7))
df['author'].value_counts().plot.pie(explode=[0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('df1 author Count')
ax[0].set_ylabel('Count')
sns.countplot('author',data=df,ax=ax[1])
ax[1].set_title('df1 author Count')
plt.show()
#Description des variables quantitatives 
df.select_dtypes(exclude='object').describe()


#description des variables quantitatives 
df.select_dtypes(include='object').describe()



df2=df[['retweet_count', 'favorite_count', 'author']]
df2['author'] = df2['author'].astype('object')
df2['author'][df2['author'] == 1 ] = 'Trump'
df2['author'][df2['author'] == 0 ] = 'Clinton'
fig=plt.gcf()
fig.set_size_inches(10,7)
fig=sns.stripplot(x='author',
                  y='favorite_count',data=df2,jitter=True,
                  edgecolor='gray',size=8,palette='winter',orient='v')

df2=df[['retweet_count', 'favorite_count', 'author']]
df2['author'] = df2['author'].astype('object')
df2['author'][df2['author'] == 1 ] = 'Trump'
df2['author'][df2['author'] == 0 ] = 'Clinton'
fig=plt.gcf()
fig.set_size_inches(10,7)
fig=sns.stripplot(x='author',
                  y='retweet_count',data=df2,jitter=True,
                  edgecolor='gray',size=8,palette='winter',orient='v')


#Allos nous allons définir un new data appelé data qui contient uniquement lentgh, text et author 
data=df[['author', 'text', 'favorite_count']] 
data['author'][data['author'] == 1 ] = 'Trump'
data['author'][data['author'] == 0 ] = 'Clinton'



data.sample(5)



#Regardons est ce que la taille des twett peut nous indiquer sa  provenence 
sns.factorplot('author','favorite_count',data=df)
plt.ioff()
plt.show()



#Regardone la distribution des length en fonction des candidants 
mpl.rcParams['patch.force_edgecolor'] = True
plt.style.use('seaborn-bright')
df.hist(column='favorite_count', by='author', bins=50,figsize=(11,5))



import nltk
from wordcloud import WordCloud,STOPWORDS
import re
from nltk.corpus import stopwords
#nltk.download('stopwords')

def cleanedWords(raw_sentence):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_sentence)
    words = letters_only.lower().split()                            
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words
                       if w not in stops]
    return meaningful_words


#sms=tweet
#création de la base tweet
tweet=data[['text','author']]
#fonction de code en binaire 0/1
def transformSpamColumn(x):
    if x=='Clinton':
        return 0
    return 1
#appication de la fonction sur tweet['author']
tweet['author']=tweet['author'].apply(transformSpamColumn)

def getWordCloud(data, author):
    data=data[data['author'] == author]
    words = ' '.join(df['text'])
    cleaned_word = " ".join(cleanedWords(words))
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=3000,
                      height=2500
                     )
    wordcloud.generate(cleaned_word)
    plt.figure(1,figsize=(12, 12))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
    

#les mots les plus fréquent de Trump
getWordCloud(tweet,1)

#Les mots plus fréquents de Clinton
getWordCloud(tweet,0)

 ## Partie Modélisation¶


In [ ]:
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
#from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier 

import calendar
import datetime
import re
messages = df[['author','text']]
test = xtest[['text']] 
messages.columns = ['label', 'message']
test.columns = ['message']


print(messages[:5])



def split_into_tokens(message):
    message = message  # convert bytes into proper unicode
    return TextBlob(message).words



messages.message.head()

messages.message.apply(split_into_tokens).head()


# nltk.download('wordnet')
def split_into_lemmas(message):
    message = message.lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

messages.message.apply(split_into_lemmas).head()



bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(messages['message'])
print(len(bow_transformer.vocabulary_))
print(bow_transformer.get_feature_names()[:5])

messages_bow = bow_transformer.transform(messages['message'])
print('sparse matrix shape:', messages_bow.shape)
print('number of non-zeros:', messages_bow.nnz)
print('sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1])))


tfidf_transformer = TfidfTransformer().fit(messages_bow)



print (tfidf_transformer.idf_[bow_transformer.vocabulary_['the']])
print (tfidf_transformer.idf_[bow_transformer.vocabulary_['hannity']])

messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)


 messages['label'] = messages['label']
 messages['label'] =  messages['label'].astype('int64')



spam_detector = SVC(kernel='sigmoid', gamma=1.0).fit(messages_tfidf,  messages['label'])



all_predictions = spam_detector.predict(messages_tfidf)



tr_acc = accuracy_score(messages['message'], all_predictions)
#print("Accuracy on training set:  %.2f%%" % (100 * tr_acc))

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', SVC(kernel='sigmoid', gamma=1.0)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])   
scores = cross_val_score(pipeline,  # steps to convert raw messages into models
                         messages['message'], # training data
                         messages['label'],  # training labels
                         cv=10,  # split data randomly into 10 parts: 9 for training, 1 for scoring
                         scoring='accuracy',  # which scoring metric?
                         n_jobs=-1,  # -1 = use all cores = faster
                         )
print(scores)

print(classification_report(messages['label'], all_predictions))

print('Mean score:', scores.mean(), '\n')
print('Stdev:', scores.std())

skf = StratifiedKFold(n_splits=5)
params = {
    'tfidf__use_idf': (True, False),
    'bow__analyzer': (split_into_lemmas, split_into_tokens),
}

grid = GridSearchCV(
    pipeline,  # pipeline from above
    params,  # parameters to tune via cross validation
    refit=True,  # fit using all available data at the end, on the best found param combination
    n_jobs=-1,  # number of cores to use for parallelization; -1 for "all cores"
    scoring='accuracy',  # what score are we optimizing?
    cv=StratifiedKFold( n_splits=5),  # what type of cross validation to use
)

%time nb_detector = grid.fit(messages['message'], messages['label'])
print(nb_detector.cv_results_)


from sklearn.model_selection import GridSearchCV
nb_detector.cv_results_


predictions = nb_detector.predict(test['message'])

print(predictions.shape[0])



predictions = pd.Series(predictions)
predictions.index = range(predictions.shape[0])
predictions.to_csv("predictions.csv", index_label ="index", header = ["prediction"])





# AUTRE EXEMPLE


In [5]:
!pip install nltk

     |████████████████████████████████| 1.4MB 886kB/s eta 0:00:01
     |████████████████████████████████| 92kB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 675kB 936kB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.6MB/s eta 0:00:01
  Stored in directory: /home/sid31/snap/jupyter/6/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [9]:
!pip install pandas

     |████████████████████████████████| 9.9MB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 512kB 1.1MB/s eta 0:00:01


In [14]:
# Importing necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus# sample text for performing tokenization
from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences


In [12]:
text = "In Brazil they drive on the right-hand side of the road. Brazil has a large coastline on the eastern side of South America"
# importing word_tokenize from nltk

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1056)>


False

In [19]:
token = word_tokenize(text)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/sid31/snap/jupyter/6/nltk_data'
    - '/snap/jupyter/6/nltk_data'
    - '/snap/jupyter/6/share/nltk_data'
    - '/snap/jupyter/6/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


#### Finding frequency distinct in the text

In [6]:

# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist
from nltk.probability import FreqDist
fdist = FreqDist(token)
fdist

# To find the frequency of top 10 words
fdist1 = fdist.most_common(10)
fdist1


# Importing Porterstemmer from nltk library
# Checking for the word ‘giving’ 
from nltk.stem import PorterStemmer
pst = PorterStemmer()
pst.stem('waiting')


# Checking for the list of words
stm = ["waited", "waiting", "waits"]
for word in stm :
    print(word+ ":" +pst.stem(word))


# Importing LancasterStemmer from nltk
from nltk.stem import LancasterStemmer
lst = LancasterStemmer()
stm = ['giving', 'given', 'given', 'gave']
for word in stm :
    print(word+ ":" +lst.stem(word))

NameError: name 'token' is not defined